In [37]:
import parse

In [38]:
with open("input.txt") as file:
    sensor_lines = file.read().split("\n")
    target_y = 2000000
    max_y_grid = 4000000
    max_x_grid = 4000000

In [25]:
sensor_lines = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""".split("\n")
target_y = 10
max_y_grid= 20
max_x_grid =20

In [39]:
def get_distance(sensor, beacon):
    distance = abs(sensor[0]- beacon[0]) + abs(sensor[1]-beacon[1])
    return distance

In [40]:
sensors_and_beacons = []
beacons_on_target = set()
for sensor_line in sensor_lines:
    positions = parse.parse("Sensor at x={}, y={}: closest beacon is at x={}, y={}", sensor_line)
    sensor = (int(positions[0]), int(positions[1]))
    beacon = (int(positions[2]), int(positions[3]))
    if beacon[1] == target_y:
        beacons_on_target.add(beacon)
    distance = get_distance(sensor, beacon)
    sensors_and_beacons.append([sensor, beacon, distance])

In [41]:
import math
from tqdm import tqdm

In [42]:
scanned_x = set()
for sensor_beacon in tqdm(sensors_and_beacons):
    sensor = sensor_beacon[0]
    max_distance = sensor_beacon[2]
    if abs(target_y - sensor[1]) <= max_distance:
        remain_scan = max_distance - abs(target_y - sensor[1])
        min_x = sensor[0] - remain_scan
        max_x = sensor[0] + remain_scan
        [scanned_x.add(x_scans) for x_scans in range(min_x, max_x +1)]

100%|██████████| 38/38 [00:01<00:00, 25.73it/s]


In [43]:
len(scanned_x) - len(beacons_on_target)

5166077

In [64]:
def check_line(target_y):
    potential_correct = set()
    sensor_ranges = []
    complete_ranges = []
    for sensor_beacon in sensors_and_beacons:
        sensor = sensor_beacon[0]
        max_distance = sensor_beacon[2]
        if abs(target_y - sensor[1]) <= max_distance:
            remain_scan = max_distance - abs(target_y - sensor[1])
            min_x = max(sensor[0] - remain_scan,  0)
            max_x = min(sensor[0] + remain_scan, max_x_grid)
            sensor_ranges.append([min_x, max_x])
    sorted_sensors = sorted(sensor_ranges, key=lambda x: x[0])
    ranges = []
    i = 0
    for sensor in sorted_sensors:
        if len(ranges) == 0:
            ranges.append(sensor)
            continue
        if sensor[0] >= ranges[i][0] and sensor[0] <= ranges[i][1]:
            if sensor[1] >= ranges[i][1]:
                ranges[i][1] = sensor[1]
        elif sensor[1] >= ranges[i][0] and sensor[1] <= ranges[i][1]:
            if sensor[0] <= ranges[i][0]:
                ranges[i][0] = sensor[0]
        else:
            ranges.append(sensor)
            i+=1
    if len(ranges) > 1:
        print(ranges)
        print(target_y)
    return potential_correct

In [65]:
potential_correct = set()
for target_y in tqdm(range(max_y_grid+1)):
    potential_correct.union(check_line(target_y))

 68%|██████▊   | 2709354/4000001 [01:23<00:38, 33314.57it/s]

[[0, 3267800], [3267802, 4000000]]
2703981


100%|██████████| 4000001/4000001 [02:04<00:00, 32150.04it/s]


In [60]:
len(potential_correct)

0

In [66]:
3267801 * 4000000 + 2703981

13071206703981